# A Simple Calculator

This file shows the implementation of a *symbolic calculator* using `Ply`.  The grammar for the language implemented by this parser is as follows:
$$
\begin{array}{lcl}
  \texttt{stmnt}   & \rightarrow & \;\texttt{IDENTIFIER}\; \texttt{':='} \; \;\texttt{expr}\; \texttt{';'} \\
                   & \rightarrow & \;\texttt{expr}\; \texttt{';'} \\[0.2cm]   
  \texttt{expr}    & \rightarrow & \;\texttt{expr}\; \texttt{'+'} \; \texttt{product}  \\
                   & \mid        & \;\texttt{expr}\; \texttt{'-'} \; \texttt{product}  \\
                   & \mid        & \;\texttt{product}                                  \\[0.2cm]
  \texttt{product} & \rightarrow & \;\texttt{product}\; \texttt{'*'} \;\texttt{factor} \\
                   & \mid        & \;\texttt{product}\; \texttt{'/'} \;\texttt{factor} \\
                   & \mid        & \;\texttt{factor}                                   \\[0.2cm]
  \texttt{factor}  & \rightarrow &   \texttt{'('} \; \texttt{expr} \;\texttt{')'}      \\
                   & \mid        & \;\texttt{NUMBER}                                   \\
                   & \mid        & \;\texttt{IDENTIFIER}                               
\end{array}
$$

## Specification of the Scanner

In [1]:
import ply.lex as lex

We only need to define four tokens. 
The other tokens, namely the operators `+`, `-`, `*`, and `/` and the parenthesis consist 
only of a single character and can therefore be defined as literals.

In [2]:
tokens = [ 'NUMBER', 'IDENTIFIER', 'ASSIGN' ]

The token `NUMBER` specifies a *floating point number*.

In [3]:
def t_NUMBER(t):
    r'(0|[1-9][0-9]*)(\.[0-9]*)?'
    t.value = float(t.value)
    return t

In [4]:
t_IDENTIFIER = r'[a-zA-Z][a-zA-Z0-9_]*'

In [5]:
t_ASSIGN = r':='

In [6]:
def t_LINEBREAK(t):
    r'\n'
    t.lexer.lineno += 1

In [7]:
literals = ['+', '-', '*', '/', '(', ')', ';']

Blanks and tabulators are ignored.

In [8]:
t_ignore  = ' \t'

Unkown characters are reported as lexical errors.

In [9]:
def t_error(t):
    print(f"Illegal character '{t.value[0]}' at character number {t.lexer.lexpos} in line {t.lexer.lineno}.")
    t.lexer.skip(1)

The next line is necessary because we use `Ply` from a notebook.

In [10]:
__file__ = 'main'

We generate the lexer.

In [11]:
lexer = lex.lex()

## Specification of the Parser

In [12]:
import ply.yacc as yacc

The *start variable* of our grammar is `statement`.

In [13]:
start = 'stmnt'

The variable `Names2Values` is a global dictionary that stores the values that are associated with the variables.

In [14]:
Names2Values = {}

The function `p_stmnt_assign` implements the following grammar rule:
$$ \texttt{stmnt} \rightarrow \texttt{IDENTIFIER}\; \texttt{':='} \; \texttt{expr} \; \texttt{';'}$$

In [15]:
def p_stmnt_assign(p):
    "stmnt : IDENTIFIER ASSIGN expr ';'"
    Names2Values[p[1]] = p[3]

The function `p_stmnt_expr` implements the following grammar rule:
$$ \texttt{stmnt} \rightarrow \texttt{expr} \; \texttt{';'} $$

In [16]:
def p_stmnt_expr(p):
    "stmnt : expr ';'"
    print(p[1])

An *expr* is a sequence of *products* that are combined with the operator `+`.
The corresponding grammar rules are:
```
    expr : expr '+' product
         | expr '-' product
         | product
         ;
```

In [17]:
def p_expr_plus(p):
    "expr : expr '+' product"
    p[0] = p[1] + p[3]

def p_expr_minus(p):
    "expr : expr '-' product"
    p[0] = p[1] - p[3]
    
def p_expr_product(p):
    "expr : product"
    p[0] = p[1]

A *product* is a sequence of *factors* that are combined with the operator `*`.
The corresponding grammar rules are:
```
    product : product '*' factor
            | product '/' factor
            | factor
            ;
```

In [18]:
def p_product_multiply(p):
    "product : product '*' factor"
    p[0] = p[1] * p[3]

def p_product_divide(p):
    "product : product '/' factor"
    p[0] = p[1] / p[3]
    
def p_product_factor(p):
    "product : factor"
    p[0] = p[1]

A factor is either an expression in parenthesis or a number.
```
   factor : '(' expr ')'
          | NUMBER
          | IDENTIFIER
          ;
```

In [19]:
def p_factor_group(p):
    "factor : '(' expr ')'"
    p[0] = p[2]

def p_factor_number(p):
    "factor : NUMBER"
    p[0] = p[1]

def p_factor_identifier(p):
    "factor : IDENTIFIER"
    p[0] = Names2Values[p[1]]

The method `p_error` is called if a syntax error occurs.  The argument `p` is the token that could not be read.  If `p` is `None` then the syntax error occurs at the end of the input.

In [20]:
def p_error(p):
    if p:
        print(f"Syntax error at character number {p.lexer.lexpos} at token '{p.value}' in line {p.lexer.lineno}.")
    else:
        raise(f"Incomplete input.")

In [22]:
parser = yacc.yacc(write_tables=False, debug=True)

Generating LALR tables


Let's look at the action table that is generated.

The parser is invoked by calling the method `yacc.parse(s)` where `s` is a string that is to be parsed.

In [23]:
def main():
    while True:
        s = input('calc> ')
        if s == '':
            break
        yacc.parse(s)

In [24]:
main()

calc>  a := 3 * 3;
calc>  b:= 4 * 4;
calc>  z := a + b;
calc>  z;


25.0


calc>  


In [25]:
Names2Values

{'a': 9.0, 'b': 16.0, 'z': 25.0}